In [131]:

import sys
import string
import numpy as np
import pandas as pd 

# trainfile = sys.argv[1]
# testfile = sys.argv[2]
# testpredfile = sys.argv[3]

trainfile = "traindata.csv"
testfile = "testdata.csv"
testpredfile = "posneg.txt"




trainD = np.array(pd.read_csv(trainfile , header=0))
testD = np.array(pd.read_csv(testfile, header=0))

# trainX = trainD[:,0]
# trainY = trainD[:,1]
trainExamples = trainD.shape[0]

In [132]:
print(trainD.shape)
print(trainD[0])
# print(tr)
print(np.unique(trainD[:,1]))

# one = np.unique(trainD[:,1])[1]
# zero = np.unique(trainD[:,1])[0]
one = 'positive'
zero = 'negative'

(10000, 2)
["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality or unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts or face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish or more....so scuffles, death stares, dodgy dealings or shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other 

In [137]:
# A list of common english words which should not affect predictions
stopwords = ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone',
             'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount',
             'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around',
             'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before',
             'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both',
             'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de',
             'describe', 'detail', 'did', 'do', 'does', 'doing', 'don', 'done', 'down', 'due', 'during', 'each', 'eg',
             'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone',
             'everything', 'everywhere', 'except', 'few', 'fifteen', 'fify', 'fill', 'find', 'fire', 'first', 'five', 'for',
             'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had',
             'has', 'hasnt', 'have', 'having', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed',
             'interest', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 'last', 'latter', 'latterly', 'least', 'less',
             'ltd', 'made', 'many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine', 'more', 'moreover', 'most', 'mostly',
             'move', 'much', 'must', 'my', 'myself', 'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine',
             'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once',
             'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own',
             'part', 'per', 'perhaps', 'please', 'put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed', 'seeming',
             'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'sincere', 'six', 'sixty', 'so', 
             'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'system',
             't', 'take', 'ten', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there',
             'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'thickv', 'thin', 'third', 'this',
             'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'to', 'together', 'too', 'top', 'toward',
             'towards', 'twelve', 'twenty', 'two', 'un', 'under', 'until', 'up', 'upon', 'us', 'very', 'via', 'was', 'we',
             'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby',
             'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom',
             'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours', 'yourself',
             'yourselves']


In [133]:


#preprocess the training data
vocab = {}
total = {}
vocab[one] = {}
vocab[zero]= {}
vocab["full"]={}
total[one]={}
total[zero]={}
total[one]["words"] = 0 
total[one]["examples"] = 0 
total[zero]["words"] = 0 
total[zero]["examples"] = 0 

from nltk.stem import PorterStemmer 
ps = PorterStemmer()
# vocabulary table contains 2 dictionaries positive and negative
# which contains frequencies of words corresponding to that review type 
# total table contains the sum of frequencies of all words belonging to that review type
for i in range(trainExamples):
    review = trainD[i][1]
    for w in trainD[i][0].split():
        word  = w.lower()
        word = ps.stem(word)
        if (len(word)>2):
            if word not in vocab["full"]:
                vocab["full"][word]=1
            
            if word in vocab[review]:
                vocab[review][word]+=1
            else:
                vocab[review][word]=1
        total[review]["words"]+=1
    total[review]["examples"] +=1   

for word in stopwords:
    if word in vocab[one]:
        vocab[one][word]=0
    if word in vocab[zero]:
        vocab[zero][word]=0
    

lenvocab = len(vocab["full"])
reviewProb = {}
reviewProb[one] = np.log(total[one]["examples"]/(total[one]["examples"] + total[zero]["examples"]))
reviewProb[zero] = np.log(total[zero]["examples"]/(total[one]["examples"] + total[zero]["examples"]))

print("Train done")

Train done


In [134]:
#finds absolute of log of probability of X given Y
def probXgivenY(X,Y):
    logsum = 0;
    for w in X.split():
        word  = w.lower()
        word = ps.stem(word)
        if word in vocab[Y]:
            logsum += (np.log(vocab[Y][word]+1/(total[Y]["words"] + lenvocab)))
            
    return logsum

def predict(X):
    pos = probXgivenY(X,one) + reviewProb[one]
    neg = probXgivenY(X,zero) + reviewProb[zero]
    if pos>=neg:
        return one
    else:  
        return zero

In [135]:
# sum=0
# for i in range(trainExamples):
#     if trainD[i,1]==predict(trainD[i,0]):
#         sum+=1
# print(sum/trainExamples)
# print(trainD[222,1])
# print(predict(trainD[222,0]))
# # 
testexamples = testD.shape[0]
print(testexamples)

0.921
positive
positive
40000


In [113]:
# preds = testD[:,0]
preds = np.zeros(testexamples)
print(preds[0])

0.0


In [114]:
print(zero)

negative


In [115]:

for i in range(testexamples):
    prediction = predict(testD[i,0])
    if prediction == zero:
        preds[i]=0
    else:
        preds[i]=1
    
    if (i+1)%200==0:
        print(i+1)

In [98]:
# print(preds[0])
np.savetxt(testpredfile,preds)


negative
